## Answearing project

In [2]:
import bs4 as bs
import urllib.request
import re
import nltk
import pickle
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import datetime
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI)

C:\Users\Mag\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Klasa Answear


In [18]:
class Answear():
    
    def __init__(self):
        self.text_base = ''
                
    def create_model(self, subj: str):
        
        self.gather_data(subj)

        if len(self.text_base)==0:
            raise Exception('No text for model to learn')
         

        self.text_base = self.process_text(self.text_base)

        sentences = nltk.sent_tokenize(self.text_base)
        sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
        stop_list = stopwords.words('english')
        for i in range(len(sentences)):
            sentences[i] = [word for word in sentences[i] if word not in stop_list]

        return Word2Vec(sentences, min_count=3)
            
       
    def gather_data(self, subj: str):
        
        print('Gathering data')
        page_start = wiki_wiki.page(subj.lower())
            
        assert (page_start.exists()==1), 'Sorry, no information about this subject in wikipedia...'
        
        self.text_base = page_start.text
        links = page_start.links
        
        if len(links.keys())!=0:
            i, j =1, 0
            time_start = datetime.datetime.now()
            print('Pages to read: ', len(links.keys()))
            
            for link in links.keys(): 
                self.text_base += wiki_wiki.page(link).text
                if round(100*i/len(links.keys()),0)>=j+5:
                    j=round(100*i/len(links.keys()),0)
                    print('Current progress: {:.0f}% pages.'.format(j))
                i+=1
            time_stop = datetime.datetime.now()
            print('Total time: ', time_stop-time_start)
    
    
     
    def what_is_the_meaning_of(self, question: str = 'life', according_to: str = 'monthy python'):
        print('Checking existing models...')
        try:
            with open('models.pickle','rb') as f:
                models_dict = pickle.load(f)
        except Exception as e:
            print('No models stored yet.')
            models_dict={}
            
        if according_to in models_dict.keys():
            self.model = models_dict[according_to]
            print('Model loaded.')
        else:
            print('New model will be created.')
            self.model = self.create_model(according_to)
            models_dict[according_to] = self.model
            with open('models.pickle','wb') as f:
                pickle.dump(models_dict,f)
                
        print('The meaning of {} according to {} is : {}, {} or {}.'.format(question, according_to, self.model.wv.most_similar(question)[0][0],self.model.wv.most_similar(question)[1][0],self.model.wv.most_similar(question)[2][0]) )
    
    @staticmethod
    def process_text(text: str):
        
        text = re.sub(r"[\[\]=,.!?\-'()\"]", " ", text)
        text = re.sub(r"<[a-z/]+>", " ", text)
        text = text.lower()
        text = re.sub(r'\d', ' ', text)
        text = re.sub(r"\s+", " ", text)

        return text

In [19]:
test = Answear()

In [5]:
test.what_is_the_meaning_of('cat', according_to = 'dog') 

Checking existing models...
New model will be created.
Gathering data
Pages to read:  876
Current progress: 5% pages.
Current progress: 10% pages.
Current progress: 15% pages.
Current progress: 20% pages.
Current progress: 25% pages.
Current progress: 30% pages.
Current progress: 35% pages.
Current progress: 40% pages.
Current progress: 45% pages.
Current progress: 50% pages.
Current progress: 55% pages.
Current progress: 60% pages.
Current progress: 65% pages.
Current progress: 70% pages.
Current progress: 75% pages.
Current progress: 80% pages.
Current progress: 85% pages.
Current progress: 90% pages.
Current progress: 95% pages.
Current progress: 100% pages.
Total time:  

0:07:40.416402
The meaning of cat according to dog is : brucella, doctrinal or pet.


In [5]:
test.what_is_the_meaning_of('brian', according_to = 'monty python') 

Checking existing models...
Model loaded.
The meaning of brian according to monty python is : life, flying or monty.


In [20]:
test.what_is_the_meaning_of('tree', according_to = 'lkjh') 

Checking existing models...
New model will be created.
Gathering data


AssertionError: Sorry, no information about this subject in wikipedia...

In [11]:
with open('models.pickle','rb') as f:
    models_dict = pickle.load(f)
        

In [13]:
with open('models.pickle','wb') as f:
    pickle.dump(models_dict,f)

In [12]:
del models_dict['lkjh']